### Training the Q model



In [23]:
# the required imports
import numpy as np
import pandas as pd
# from linear_aproximation import Model
# from environment import network
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# additional import for the Neural Network
from keras.layers import Dense
from keras.models import Sequential, load_model
from keras import optimizers
import tensorflow as tf


In [2]:
# path = '/Users/mawongh/OneDrive/REFERENCE FILE/D/Disertation/brainstorming/'
path = '/home/mawongh/ws/datasets/'
dataset = pd.read_pickle(path + 'full_dataset.pickle')
dataset.tail(3)


,action,reward,s_next,state
299475,121.0,-2.915486,"[2733, 2733, 2733, 3200, 3200, 3200, 2500, 250...","[2733, 2733, 2733, 3200, 3200, 3200, 2500, 250..."
299476,121.0,-3.054551,"[2733, 2733, 2733, 3200, 3200, 3200, 2500, 250...","[2733, 2733, 2733, 3200, 3200, 3200, 2500, 250..."
299477,121.0,-2.653020,"[2733, 2733, 2733, 3200, 3200, 3200, 2500, 250...","[2733, 2733, 2733, 3200, 3200, 3200, 2500, 250..."


In [3]:
# decorrelating samples
N = len(dataset)
sample_indexes = np.random.choice(np.arange(N), size = N, replace=False)
dataset_decorr = dataset.iloc[sample_indexes]

S_woScale = dataset_decorr.state.tolist()
Snext_woScale = dataset_decorr.s_next.tolist()
a = dataset_decorr['action'].values
r = dataset_decorr['reward'].values

In [4]:
scaler = StandardScaler()
scaler.fit(S_woScale)
S = scaler.transform(S_woScale)
Snext = scaler.transform(Snext_woScale)


In [5]:
# model 6
from keras.models import load_model
modelfilename = 'MDP_model6.h5'

MDP_model = load_model(path + modelfilename)

# Q model
Q_model = Sequential()
n_cols = 105

Q_model.add(Dense(400, activation = 'relu', input_shape=(n_cols,)))
Q_model.add(Dense(400, activation = 'relu'))
Q_model.add(Dense(400, activation = 'relu'))
Q_model.add(Dense(126, activation = 'linear'))
Q_model.compile(optimizer=optimizers.Adam(), loss='mean_squared_error')


In [6]:
def train_Qmodel_1st_pass(num_epochs = 20, gamma = 0.9):
    for epoch in np.arange(num_epochs):
        print('Q model, first pass, epoch: {}'.format(epoch))
        for step in np.arange(len(S)):
            s = S[step].reshape(1,-1)
            s_next = Snext[step].reshape(1,-1)
            target = r[step] + gamma * np.max(MDP_model.predict_on_batch(s_next))
            Q_temp = Q_model.predict_on_batch(s)
            Q_temp[0,int(a[step])] = target
            Q_model.train_on_batch(s,Q_temp)
            
            

In [9]:
scaler_mean_file = 's_model-based_mean_.csv'
np.savetxt(path + scaler_mean_file, scaler.mean_, delimiter=',')

scaler_scale_file = 's_model-based_scale_.csv'
np.savetxt(path + scaler_scale_file, scaler.scale_, delimiter=',')

In [ ]:
# train the Q model with all the samples (300K)
train_Qmodel_1st_pass()
print('trained!')


Q model, first pass, epoch: 0
Q model, first pass, epoch: 1
Q model, first pass, epoch: 2
Q model, first pass, epoch: 3


In [11]:
Qmodelfilename = 'Q_model6_1stpass.h5'
Q_model.save(path + Qmodelfilename)

In [12]:
def train_Qmodel_2nd_pass(num_epochs = 20, gamma = 0.9):
    for epoch in np.arange(num_epochs):
        print('Q model, second pass, epoch: {}'.format(epoch))
        for step in np.arange(len(S)):
            s = S[step].reshape(1,-1)
            s_next = Snext[step].reshape(1,-1)
            target = r[step] + gamma * np.max(Q_model.predict_on_batch(s_next))
            Q_temp = Q_model.predict_on_batch(s)
            Q_temp[0,int(a[step])] = target
            Q_model.train_on_batch(s,Q_temp)



In [ ]:
# train the Q model with all the samples (300K)
train_Qmodel_2nd_pass()
print('trained!')


Q model, second pass, epoch: 0
Q model, second pass, epoch: 1


In [14]:
Qmodelfilename = 'Q_model6_2ndpass.h5'
Q_model.save(path + Qmodelfilename)

In [15]:
Qmodel_weigths_filename = 'Q_model6_2ndpass_weights.h5'
Q_model.save_weights(path + Qmodel_weigths_filename)

In [24]:
Qmodel_1 = load_model(path + Qmodelfilename)


In [33]:
Qmodel_1.predict(S[2].reshape(1,-1))[0]


array([-22.02394295, -23.89875603, -23.73156548, -23.91939735,
       -24.12434006, -23.26884651, -23.77721405, -23.2222271 ,
       -23.34122276, -23.72259712, -24.30957985, -23.27479935,
       -23.953619  , -23.82709694, -23.9458065 , -23.92469025,
       -23.90373802, -23.32255363, -23.58679199, -23.90897369,
       -23.83125877, -23.57485962, -23.85373306, -23.61318016,
       -23.79582405, -23.38915062, -23.30856323, -23.60995102,
       -24.16109467, -23.71725655, -23.38339615, -23.79895973,
       -23.49809456, -23.71014214, -24.05584717, -22.95737457,
       -23.82899284, -23.69297218, -23.33287811, -23.90036964,
       -24.10214615, -23.28546333, -23.72054672, -23.68875504,
       -23.20798683, -23.06004906, -23.9834919 , -23.14255714,
       -23.49912071, -23.89904976, -24.15880775, -23.48823929,
       -24.23230553, -23.75199699, -24.21553421, -23.88863182,
       -24.05475998, -23.50255775, -23.85937119, -23.36322403,
       -24.05265236, -23.92456627, -23.48961639, -23.52

In [ ]:
from sklearn.preprocessing import StandardScaler
num_of_experiments = 10
# S = np.array([x for x in study_dataset.state])

for experiments in np.arange(num_of_experiments):
    sample_indexes = [np.random.choice(np.arange(N), size = sz, replace=False)
                     for sz in sample_sizes]
    # np.random.choice(np.arange(10), size =5, replace=False)
    study_dataset = [dataset.iloc[idx] for idx in sample_indexes]
    
    
    S_woScale = [study_dataset[i].state.tolist() for i in np.arange(len(study_dataset))]
    a_r = [df[['action', 'reward']].values for df in study_dataset]
    
    scaler = StandardScaler()
    
    S = [scaler.fit_transform(S_woScale[i]) for i in np.arange(len(S_woScale))]

    
    for ds_idx in np.arange(sample_sizes_n):
        
        #splitting dataset into train/test
        S_train, S_test, a_r_train, a_r_test = train_test_split(
            S[ds_idx], a_r[ds_idx], test_size=0.15, random_state=42)
        
        a_train = a_r_train[:,0]
        a_test = a_r_test[:,0]
        
        r_train = a_r_train[:,1]
        r_test = a_r_test[:,1]
        
#         new_S_train = np.array([S_train[i] for i in np.arange(len(S_train))])
#         S_train = new_S_train
#         new_S_test = np.array([S_test[i] for i in np.arange(len(S_test))])
#         S_test = new_S_test

        
        # creates the model, it has to be every time to prevent pre-trained weights
        model6 = Sequential()
        n_cols = 105

        # model 6
        model6.add(Dense(400, activation = 'relu', input_shape=(n_cols,)))
        model6.add(Dense(400, activation = 'relu'))
        model6.add(Dense(400, activation = 'relu'))
        model6.add(Dense(126, activation = 'linear'))
        model6.compile(optimizer=optimizers.Adam(), loss='mean_squared_error')
        
        print('experiment: {}, sample_size: {}'.format(experiments,len(S[ds_idx])))
        train_model6(num_epochs = 20)
#         print('trained!')
        
        y_pred = model6.predict(S_train)
        r_hat = np.array([y_pred[i,int(a_train[i])] for i in np.arange(len(S_train))])
        train_MSE = mean_squared_error(r_train, r_hat)
        print('train MSE: {}'.format(train_MSE))
        
        y_pred = model6.predict(S_test)
        r_hat = np.array([y_pred[i,int(a_test[i])] for i in np.arange(len(S_test))])
        test_MSE = mean_squared_error(r_test, r_hat)
        print('test MSE: {}'.format(test_MSE))
        
        algorithm = 'NN Model 6'
        record = df.append({'algorithm': algorithm,
                            'sample_size': len(S[ds_idx]),
                            'mse': train_MSE,
                            'data': 'Train'},
                           ignore_index=True)
        record = record.append({'algorithm': algorithm,
                                'sample_size': len(S[ds_idx]),
                                'mse': test_MSE,
                                'data': 'Test'},
                               ignore_index=True)
        # saves the dataframe for further analysis
        record.to_csv(path + outputfile, mode='a', header=False, index=False)
        
    

In [ ]:
path + outputfile

In [ ]:
results = pd.read_csv(path + outputfile)
results.head()

In [ ]:
import seaborn as sns
plt.figure(figsize=(3,4), dpi=150)
sns.boxplot(x="data", y="mse", data=results)
plt.show()

In [34]:
-5 + 0.9*(-4)


-8.6

In [50]:
def state2stateid(s):
    return ''.join(str for str in s.astype(str).tolist())


In [51]:
state2stateid(dataset.state[0])


'273327332733320032003200250025002500426642664266376637663766666666666200020002000126612661266173317331733160016001600700700700276627662766113311331133250025002500401603408014032080220320601803404022026010012026080220300384442383636443844404040444240443640363844111000101101101101001'

## Playground

In [ ]:
np.array([[1,2], [1], [2,3]])

In [ ]:
np.random.choice(20, 30, replace=False)

In [ ]:
np.min([20, 30])

In [ ]:
100 % 100

In [ ]:
model.get_weights()

In [ ]:
s = np.array([10, 20, 40, 50])
s.reshape(1,-1)[0]

In [ ]:
model.get_weights()s.reshape(-1,1)

In [ ]:
model.save('test.h5')




In [ ]:
print("%03d" % (100,))